In [66]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
import base58
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time
import sys
sys.path.append('../../GlobalRepo/')
from global_repo import *



client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [67]:
program_id = TABLES_PROGRAM_ID
metadata_program_pubkey = METAPLEX_PROGRAM_ID


In [68]:
def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')

In [69]:
seed = bytes([125,215,212,54,213,98,45,123]*4)
payer_keypair = Keypair.from_secret_key(base58.b58decode("mH4PkhAttpFXSrntbpiJgC8J6wHYFDK5PFWcPN53hMJKziWQucux7MFiW7uXnqKsAY3XBnxUtyVKnomZnD9JxHW"))
client.request_airdrop(payer_keypair.public_key, 2*10**9)
payer_keypair.public_key

2ZHc9QxDDaJwqNEFzpAGUrvxCWZSNnXSffHxV9hG2axp

In [70]:
dsol_mint = DSOL_MINT_ID
governor_mint = PublicKey("J3ACcSXB2ACbX2CL4FWxvZh81KRQt4AtF3XhyvpxxrJK")
payer_bump = 5

payer_governor_token_address = instructions.get_associated_token_address(payer_keypair.public_key, governor_mint)

table_mint_keypair = Keypair.generate()

payer_table_associated_token_address = instructions.get_associated_token_address(payer_keypair.public_key, table_mint_keypair.public_key)

(payer_table_data_pda, payer_table_data_pda_bump) = PublicKey.find_program_address([b"table_data_pda", bytes(payer_keypair.public_key), payer_bump.to_bytes(4,"big")], program_id);

(payer_table_metadata_pda, payer_table_metadata_pda_bump) = PublicKey.find_program_address([b"metadata", bytes(metadata_program_pubkey), bytes(table_mint_keypair.public_key)], metadata_program_pubkey) 

(table_sales_pda, table_sales_nonce) = PublicKey.find_program_address([b'table_sales_pda'], program_id)

payer_dsol_token_account = instructions.get_associated_token_address(payer_keypair.public_key, dsol_mint)

(table_mint_authority, nonce) = PublicKey.find_program_address([b"table_mint_authority_pda"], program_id)

payer_table_data_pda

9tJss21NKCaFSU9CGE8wrPC8dH9mQpoJbqNpinqf8f3P

In [71]:
InstructionEnum = Enum(
    "InitTable" / CStruct("governor_reward" / U32, "payer_bump" / U32),
    "SignTableMint",
    "CreateTableSalesAccount",
    "BurnNFTs" / CStruct("rarity" / U8),
    "MintTable",
    enum_name = "InstructionEnum"
)


vault_pda, _ = PublicKey.find_program_address([b"Dsol_vault_tables"], program_id)
vault_pda_dsol_token_account_meta = AccountMeta(instructions.get_associated_token_address(vault_pda, dsol_mint), False, True)

(pending_pda, pending_nonce) = PublicKey.find_program_address([b'pending_tables'], program_id)
pending_tables_meta = AccountMeta(pending_pda, False, True)

main_vault_assoc = VAULT_DSOL_ASSOCIATED_ID

main_vault_meta = AccountMeta(main_vault_assoc, False, True)

gov_data_pubkey, _ = PublicKey.find_program_address([b'governor_data_pda', bytes(governor_mint)], GOVERNOR_PROGRAM_ID)
gove_data_meta = AccountMeta(gov_data_pubkey, False, True)

governor_mint_authority, _ = PublicKey.find_program_address([b"mint_authority"], GOVERNOR_PROGRAM_ID)
gov_mint_authority_meta = AccountMeta(governor_mint_authority, False, True)

authorizer_pda, _ = PublicKey.find_program_address([b"authorizer_pda"], program_id)
authorizer_meta = AccountMeta(authorizer_pda, False, True)

In [72]:

payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)
payer_dsol_token_account_meta = AccountMeta(payer_dsol_token_account, False, True)
payer_governor_token_account_meta = AccountMeta(payer_governor_token_address, False, True)
# payer_table_associated_account_meta = AccountMeta(payer_table_associated_token_address, False, True)
payer_table_data_pda_meta = AccountMeta(payer_table_data_pda, False, True)
payer_table_metadata_meta = AccountMeta(payer_table_metadata_pda, False, True)
# vault_account_meta = AccountMeta(vault_keypair.public_key, False, True)
# vault_dsol_token_account_meta = AccountMeta(vault_dsol_token_account, False, True)
table_mint_account_meta = AccountMeta(table_mint_keypair.public_key, True, True)
table_mint_authority_meta = AccountMeta(table_mint_authority, False, False)
table_sales_pda_meta = AccountMeta(table_sales_pda, False, True)

rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
sysvar_clock_meta = AccountMeta(solana.sysvar.SYSVAR_CLOCK_PUBKEY, False, False)
associated_program_meta = AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False)
metadata_meta = AccountMeta(metadata_program_pubkey, False, False)
dsol_mint_meta = AccountMeta(dsol_mint, False, False)
compute_program_meta = AccountMeta(PublicKey("ComputeBudget111111111111111111111111111111"), False, False)
gov_account_meta = AccountMeta(governor_mint, False, False)
vault_account_meta = AccountMeta(vault_pda, False, False)


In [73]:
def init_table_mint():
    accounts = [
        payer_dsol_token_account_meta,
        payer_governor_token_account_meta,
        gov_account_meta,
        gov_mint_authority_meta,
        gove_data_meta,
        payer_table_data_pda_meta,
        vault_account_meta,
        vault_pda_dsol_token_account_meta,
        spl_program_meta,
        sysvar_clock_meta,
        authorizer_meta,

        AccountMeta(GOVERNOR_PROGRAM_ID, False, False),
        spl_program_meta,
    ]

    # print(accounts)
    transaction = Transaction()
    transaction.add(
        TransactionInstruction(
            accounts,
            program_id,
            InstructionEnum.build(InstructionEnum.enum.SignTableMint())
        )
    )
    
    return f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair)['result']}?cluster=devnet"

init_table_mint()

# time.sleep(5)

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: invalid instruction data', 'data': {'accounts': None, 'err': {'InstructionError': [0, 'InvalidInstructionData']}, 'logs': ['Program 7fmmvCJ1uH4sfLCdZrDAjJnfHejwtFbYL7aG8nnNR6Yz invoke [1]', 'Program log: Reached checkpoint 1', 'Program log: Governor is frozen', 'Program 7fmmvCJ1uH4sfLCdZrDAjJnfHejwtFbYL7aG8nnNR6Yz consumed 4618 of 200000 compute units', 'Program 7fmmvCJ1uH4sfLCdZrDAjJnfHejwtFbYL7aG8nnNR6Yz failed: invalid instruction data'], 'unitsConsumed': 0}}

In [ ]:
time.sleep(5)
accountSchema = CStruct(
    "date_created" / U32,
    "id" / U32,
    "rarity" / U8,
    "creators" / Vec(U8[32]),
    "num_creators" / U8,
    "governor_reward" / U32
)

getAccountInfo(client, payer_table_data_pda)


Container(date_created=1646673379, id=4, rarity=10, creators=ListContainer([ListContainer([118, 170, 80, 9, 240, 239, 203, 27, 117, 25, 65, 80, 67, 180, 52, 12, 39, 96, 27, 251, 122, 104, 168, 253, 198, 184, 97, 218, 241, 77, 127, 219])]), num_creators=5, governor_reward=500)